In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

In [5]:
file_beer_sales_1=pd.read_csv('./data/2-4-1-beer-sales-1.csv')

In [6]:
file_beer_sales_1.head(3)

,sales
0,87.47
1,103.67
2,83.29


In [7]:
# サンプルサイズ
print(len(file_beer_sales_1))

100


In [10]:
# listにまとめる
data_list=file_beer_sales_1['sales'].values.tolist()
print(data_list)

[87.47, 103.67, 83.29, 131.91, 106.59, 83.59, 109.75, 114.77, 111.52, 93.89, 130.24, 107.8, 87.58, 55.71, 122.5, 99.1, 99.68, 118.88, 116.42, 111.88, 118.38, 115.64, 101.49, 60.21, 112.4, 98.88, 96.88, 70.58, 90.44, 108.36, 127.17, 97.94, 107.75, 98.92, 72.46, 91.7, 92.11, 98.81, 122.0, 115.26, 96.71, 94.93, 113.94, 111.13, 86.22, 85.85, 107.29, 115.37, 97.75, 117.62, 107.96, 87.76, 106.82, 77.41, 128.66, 139.61, 92.66, 79.12, 111.39, 97.3, 148.03, 99.22, 113.79, 100.56, 85.13, 103.78, 63.9, 129.31, 103.07, 143.45, 109.51, 85.8, 112.21, 81.32, 74.93, 105.83, 91.13, 100.02, 101.49, 88.21, 88.63, 97.3, 123.56, 69.53, 111.88, 106.66, 121.26, 93.92, 107.4, 105.34, 89.15, 124.16, 123.21, 114.0, 131.74, 111.17, 74.47, 88.53, 75.51, 90.53]
